In [33]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [34]:
# Carregar o dataset Titanic diretamente da URL
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Exibir as primeiras linhas do dataset para inspeção
print(df.head())

# Separar os dados de entrada (X) e a variável alvo (y)
X = df.drop(columns=["Survived", "Name", "Ticket", "Cabin", "Embarked"])  # Remover variáveis não necessárias
y = df["Survived"]

# Codificar variáveis categóricas (por exemplo, 'Sex') com LabelEncoder
label_encoder = LabelEncoder()
X["Sex"] = label_encoder.fit_transform(X["Sex"])

# Verificar valores ausentes
print("\nValores ausentes por coluna:")
print(X.isnull().sum())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  

V

In [35]:
# 1. Imputação de valores numéricos com a média
X_numeric = X.select_dtypes(include=[np.number])  # Selecionar apenas colunas numéricas
mean_imputer = SimpleImputer(strategy="mean")
X_numeric_imputed = mean_imputer.fit_transform(X_numeric)

In [36]:
# 2. Imputação de valores numéricos com a moda (valor mais frequente)
mode_imputer = SimpleImputer(strategy="most_frequent")
X_mode_imputed = mode_imputer.fit_transform(X_numeric)

In [37]:
# 2. Imputação de valores categóricos com a moda (valor mais frequente)
X_categorical = X.select_dtypes(exclude=[np.number])

# Verificar se existem colunas categóricas
if not X_categorical.empty:
    categorical_imputer = SimpleImputer(strategy="most_frequent")
    X_categorical_imputed = categorical_imputer.fit_transform(X_categorical)
else:
    X_categorical_imputed = np.array([])

# Juntar os dados imputados
if X_categorical_imputed.size > 0:
    X_imputed = pd.concat([pd.DataFrame(X_numeric_imputed, columns=X_numeric.columns),
                           pd.DataFrame(X_categorical_imputed, columns=X_categorical.columns)], axis=1)
else:
    X_imputed = pd.DataFrame(X_numeric_imputed, columns=X_numeric.columns)

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


In [38]:
# 3. Imputação de valores categóricos com a moda (valor mais frequente)
# Garantir que estamos selecionando colunas categóricas
X_categorical = X.select_dtypes(exclude=[np.number])  # Selecionar apenas colunas categóricas

# Verificar se existem colunas categóricas
if not X_categorical.empty:
    categorical_imputer = SimpleImputer(strategy="most_frequent")
    X_categorical_imputed = categorical_imputer.fit_transform(X_categorical)
else:
    X_categorical_imputed = np.array([])

# Juntar os dados imputados
if X_categorical_imputed.size > 0:
    X_imputed = pd.concat([pd.DataFrame(X_numeric_imputed, columns=X_numeric.columns),
                           pd.DataFrame(X_categorical_imputed, columns=X_categorical.columns)], axis=1)
else:
    X_imputed = pd.DataFrame(X_numeric_imputed, columns=X_numeric.columns)

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


In [39]:
# 4. Treinar o modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [40]:
# 5. Fazer previsões e calcular métricas de avaliação
y_pred = model.predict(X_test)

# Avaliação do modelo
metrics = {
    "Acurácia": accuracy_score(y_test, y_pred),
    "Precisão": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1-Score": f1_score(y_test, y_pred),
}

print("\nMétricas de Avaliação do Modelo com Imputação de Média:")
for metrica, valor in metrics.items():
    print(f"{metrica}: {valor:.4f}")


Métricas de Avaliação do Modelo com Imputação de Média:
Acurácia: 0.8212
Precisão: 0.8182
Recall: 0.7297
F1-Score: 0.7714


In [41]:
# 5. Imputação com KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Usando 5 vizinhos mais próximos
X_knn_imputed = knn_imputer.fit_transform(X_imputed)

# Dividir o dataset KNN
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn_imputed, y, test_size=0.2, random_state=42)

# Treinar o modelo Random Forest com KNN imputado
model.fit(X_train_knn, y_train_knn)
y_pred_knn = model.predict(X_test_knn)

# Avaliação do modelo com KNN
metrics_knn = {
    "Acurácia": accuracy_score(y_test_knn, y_pred_knn),
    "Precisão": precision_score(y_test_knn, y_pred_knn),
    "Recall": recall_score(y_test_knn, y_pred_knn),
    "F1-Score": f1_score(y_test_knn, y_pred_knn),
}

print("\nMétricas de Avaliação do Modelo com Imputação KNN:")
for metrica, valor in metrics_knn.items():
    print(f"{metrica}: {valor:.4f}")



Métricas de Avaliação do Modelo com Imputação KNN:
Acurácia: 0.8212
Precisão: 0.8182
Recall: 0.7297
F1-Score: 0.7714


In [42]:
# 6. Imputação com KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Usando 5 vizinhos mais próximos
X_knn_imputed = knn_imputer.fit_transform(X_imputed)

# Dividir o dataset KNN
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn_imputed, y, test_size=0.2, random_state=42)

# Treinar o modelo Random Forest com KNN imputado
model.fit(X_train_knn, y_train_knn)
y_pred_knn = model.predict(X_test_knn)

# Avaliação do modelo com KNN
metrics_knn = {
    "Acurácia": accuracy_score(y_test_knn, y_pred_knn),
    "Precisão": precision_score(y_test_knn, y_pred_knn),
    "Recall": recall_score(y_test_knn, y_pred_knn),
    "F1-Score": f1_score(y_test_knn, y_pred_knn),
}

print("\nMétricas de Avaliação do Modelo com Imputação KNN:")
for metrica, valor in metrics_knn.items():
    print(f"{metrica}: {valor:.4f}")



Métricas de Avaliação do Modelo com Imputação KNN:
Acurácia: 0.8212
Precisão: 0.8182
Recall: 0.7297
F1-Score: 0.7714


In [43]:
# 7. Imputação com Iterative Imputer (MissForest-like)
missforest_imputer = IterativeImputer(random_state=42)
X_missforest_imputed = missforest_imputer.fit_transform(X_imputed)

# Dividir o dataset com Imputação Iterativa
X_train_mf, X_test_mf, y_train_mf, y_test_mf = train_test_split(X_missforest_imputed, y, test_size=0.2, random_state=42)

# Treinar o modelo Random Forest com Imputação Iterativa
model.fit(X_train_mf, y_train_mf)
y_pred_mf = model.predict(X_test_mf)

# Avaliação do modelo com Imputação Iterativa
metrics_mf = {
    "Acurácia": accuracy_score(y_test_mf, y_pred_mf),
    "Precisão": precision_score(y_test_mf, y_pred_mf),
    "Recall": recall_score(y_test_mf, y_pred_mf),
    "F1-Score": f1_score(y_test_mf, y_pred_mf),
}

print("\nMétricas de Avaliação do Modelo com Imputação Iterativa:")
for metrica, valor in metrics_mf.items():
    print(f"{metrica}: {valor:.4f}")


Métricas de Avaliação do Modelo com Imputação Iterativa:
Acurácia: 0.8212
Precisão: 0.8182
Recall: 0.7297
F1-Score: 0.7714
